# Create TensorFlow Deep Neural Network Model

**Learning Objective**
- Create a DNN model using the high-level Estimator API 

## Introduction

We'll begin by modeling our data using a Deep Neural Network. To achieve this we will use the high-level Estimator API in Tensorflow. Have a look at the various models available through the Estimator API in [the documentation here](https://www.tensorflow.org/api_docs/python/tf/estimator). 

Start by setting the environment variables related to your project.

In [1]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [2]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [4]:
%%bash
ls *.csv

eval.csv
test.csv
train.csv


In [5]:
!head train.csv

8.41945378578,False,21,Single(1),38.0
8.062304921339999,True,27,Single(1),38.0
6.1244416383599996,Unknown,37,Single(1),38.0
9.37626000286,Unknown,25,Single(1),40.0
7.5618555866,False,35,Single(1),39.0
7.48689841752,Unknown,14,Single(1),38.0
7.27084540076,False,24,Single(1),41.0
7.43839671988,False,28,Single(1),42.0
7.12313568522,Unknown,26,Single(1),40.0
7.3744626639,Unknown,29,Single(1),36.0


## Create TensorFlow model using TensorFlow's Estimator API ##

We'll begin by writing an input function to read the data and define the csv column names and label column. We'll also set the default csv column values and set the number of training steps.

In [6]:
import shutil
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.15.0


#### **Exercise 1**

To begin creating out Tensorflow model, we need to set up variables that determine the csv column values, the label column and the key column. Fill in the TODOs below to set these variables. Note, `CSV_COLUMNS` should be a list and `LABEL_COLUMN` should be a string. It is important to get the column names in the correct order as they appear in the csv train/eval/test sets. If necessary, look back at the previous notebooks at how these csv files were created to ensure you have the correct ordering. 

We also need to set `DEFAULTS` for each of the CSV column values we prescribe. This will also the a list of entities that will vary depending on the data type of the csv column value. Have a look back at the previous examples to ensure you have the proper formatting.

In [7]:
# Determine CSV, label, and key columns
CSV_COLUMNS = ['weight_pounds','is_male','mother_age','plurality','gestation_weeks']
LABEL_COLUMN = 'weight_pounds'

# Set default values for each CSV column
DEFAULTS = [[0.0],['Null'],[0],['Null'],[0.0]]
TRAIN_STEPS = 1000

### Create the input function

Now we are ready to create an input function using the Dataset API.

#### **Exercise 2**

In the code below you are asked to complete the TODOs to create the input function for our model. Look back at the previous examples we have completed if you need a hint as to how to complete the missing fields below. 

In the first block of TODOs, your `decode_csv` file should return a dictionary called `features` and a value `label`.

In the next TODO, use `tf.gfile.Glob` to create a list of files that match the given `filename_pattern`. Have a look at the documentation for `tf.gfile.Glob` if you get stuck.

In the next TODO, use `tf.data.TextLineDataset` to read text file and apply the `decode_csv` function you created above to parse each row example. 

In the next TODO you are asked to set up the dataset depending on whether you are in `TRAIN` mode or not. (**Hint**: Use `tf.estimator.ModeKeys.TRAIN`). When in `TRAIN` mode, set the appropriate number of epochs and shuffle the data accordingly. When not in `TRAIN` mode, you will use a different number of epochs and there is no need to shuffle the data. 

Finally, in the last TODO, collect the operations you set up above to produce the final `dataset` we'll use to feed data into our model. 

Have a look at the examples we did in the previous notebooks if you need inspiration.

In [8]:
# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API
def read_dataset(filename_pattern, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(line_of_text):
            columns = tf.decode_csv(records=line_of_text, record_defaults=DEFAULTS)
            features = columns[1:]
            label = columns[0]
            return features, label
    
        # Create list of files that match pattern
        file_list = tf.gfile.Glob(file_pattern = filename_pattern)

        # Create dataset from file list
        dataset = dataset.flat_map(map_func = lambda filename:tf.data.TextLineDataset(filenames = file_list))
        dataset = dataset.map(map_func = decode_csv)
     
        # In training mode, shuffle the dataset and repeat indefinitely
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size = 10 * batch_size).repeate(count=False).batch(batch_size = batch_size)
        else:
            dataset = dataset.shuffle(buffer_size = 10 * batch_size).repeate(count=1).batch(batch_size = batch_size)
            
        # This will now return batches of features, label
        return dataset
    return _input_fn


def read_dataset(filename_pattern, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(records = value_column, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label
    
        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename = filename_pattern)

        # Create dataset from file list
        dataset = (tf.data.TextLineDataset(filenames = file_list)  # Read text file
                     .map(map_func = decode_csv))  # Transform each elem by applying decode_csv fn

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(count = num_epochs).batch(batch_size = batch_size)
        return dataset
    return _input_fn

### Create the feature columns

Next, we define the feature columns

#### **Exercise 3**

There are different ways to set up the feature columns for our model. 

In the first TODO below, you are asked to create a function `get_categorical` which takes a feature name and its potential values and returns an indicator `tf.feature_column` based on a categorical with vocabulary list column. Look back at the documentation for `tf.feature_column.indicator_column` to ensure you call the arguments correctly.

In the next TODO, you are asked to complete the code to create a function called `get_cols`. It has no argumnets but should return a list of all the `tf.feature_column`s you intend to use for your model. **Hint**: use the `get_categorical` function you created above to make your code easier to read.

In [9]:
def get_categorical(name, values):
    return tf.feature_column.indicator_column(
           categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key=name, vocabulary_list=values))

def get_cols():
    # Define column types
    return [get_categorical("is_male",["False","True","Unknown"]),
            tf.feature_column.numeric_column(key="mother_age"),
            get_categorical("plurality",["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)","Multiple(2+)"]),
            tf.feature_column.numeric_column(key="gestation_weeks")
    ]

### Create the Serving Input function 

To predict with the TensorFlow model, we also need a serving input function. This will allow us to serve prediction later using the predetermined inputs. We will want all the inputs from our user.

#### **Exercise 4**
In the TODOs below, create the `feature_placeholders` dictionary by setting up the placeholders for each of the features we will use in our model. Look at the documentation for `tf.placeholder` to make sure you provide all the necessary arguments. You'll need to create placeholders for the features
- `is_male`
- `mother_age`
- `plurality`
- `gestation_weeks`
- `key`

You'll also need to create the features dictionary to pass to the `tf.estimator.export.ServingInputReceiver` function. The `features` dictionary will reference the `fearture_placeholders` dict you created above. Remember to expand the dimensions of the tensors you'll incoude in the `features` dictionary to accomodate for batched data we'll send to the model for predicitons later. 

In [10]:
def serving_input_fn():
    feature_placeholders = {
        "is_male": tf.placeholder(dtype = tf.string, shape = [None]),
        "mother_age": tf.placeholder(dtype = tf.float32, shape = [None]),
        "plurality": tf.placeholder(dtype = tf.string, shape = [None]),
        "gestation_weeks": tf.placeholder(dtype = tf.float32, shape = [None])
    }
    
    features = {
        key: tf.expand_dims(input = tensor, axis = -1)
        for key, tensor in feature_placeholders.items()
    }
    
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

### Create the model and run training and evaluation

Lastly, we'll create the estimator to train and evaluate. In the cell below, we'll set up a `DNNRegressor` estimator and the train and evaluation operations. 

#### **Exercise 5**

In the cell below, complete the TODOs to create our model for training. 
- First you must create your estimator using `tf.estimator.DNNRegressor`. 
- Next, complete the code to set up your `tf.estimator.TrainSpec`, selecting the appropriate input function and dataset to use to read data to your function during training. 
- Next, set up your `exporter` and `tf.estimator.EvalSpec`.
- Finally, pass the variables you created above to call `tf.estimator.train_and_evaluate`

Be sure to check the documentation for these Tensorflow operations to make sure you set things up correctly.

In [11]:
def train_and_evaluate(output_dir):
    EVAL_INTERVAL = 300
    run_config = tf.estimator.RunConfig(
        save_checkpoints_secs = EVAL_INTERVAL,
        keep_checkpoint_max = 3)

    estimator = tf.estimator.DNNRegressor(
                   hidden_units = [10,10],
                   feature_columns = get_cols(),
                   model_dir = output_dir,
                   config = run_config)
        
    train_spec = tf.estimator.TrainSpec(
                        input_fn = lambda: read_dataset(filename_pattern="./train.csv", mode = tf.estimator.ModeKeys.TRAIN),
                        max_steps = 500)
    
    
    exporter = tf.estimator.FinalExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)

    eval_spec = tf.estimator.EvalSpec(
                       input_fn = lambda: read_dataset(filename_pattern="./eval.csv", mode = tf.estimator.ModeKeys.EVAL),
                       steps = None,
                       start_delay_secs = 1, # wait at least N seconds before first evaluation (default 120)
                       throttle_secs = 1, # wait at least N seconds before each subsequent evaluation (default 600)
                       exporters = exporter) # export SavedModel once at the end of training
 
    tf.estimator.train_and_evaluate(estimator=estimator, train_spec=train_spec, eval_spec=eval_spec)
    
    
    
def train_and_evaluate(output_dir):
    EVAL_INTERVAL = 300
    
    run_config = tf.estimator.RunConfig(
        save_checkpoints_secs = EVAL_INTERVAL,
    keep_checkpoint_max = 3)
    
    estimator = tf.estimator.DNNRegressor(
        model_dir = output_dir,
        feature_columns = get_cols(),
        hidden_units = [64, 32],
        config = run_config)
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset("train.csv", mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = TRAIN_STEPS)
    
    exporter = tf.estimator.LatestExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset("eval.csv", mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        start_delay_secs = 60, # start evaluating after N seconds
        throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
        exporters = exporter)
        
    tf.estimator.train_and_evaluate(estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)

Finally, we train the model!

In [12]:
# Run the model
shutil.rmtree(path = "babyweight_trained_dnn", ignore_errors = True) # start fresh each time
train_and_evaluate("babyweight_trained_dnn")

INFO:tensorflow:Using config: {'_session_creation_timeout_secs': 7200, '_eval_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc32fbe1470>, '_experimental_max_worker_delay_secs': None, '_global_id_in_cluster': 0, '_evaluation_master': '', '_num_ps_replicas': 0, '_device_fn': None, '_tf_random_seed': None, '_num_worker_replicas': 1, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_distribute': None, '_log_step_count_steps': 100, '_model_dir': 'babyweight_trained_dnn', '_train_distribute': None, '_keep_checkpoint_max': 3, '_save_summary_steps': 100, '_is_chief': True, '_task_id': 0, '_protocol': None, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 300, '_master': '', '_task_type': 'worker', '_service': None, '_keep_checkpoint_every_n_hours': 10000}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and e

Look at the results of your training job above. What RMSE (`average_loss`) did you get for the final eval step?

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License